# Wildfire Hazard Assessment using Machine Learning and Remote Sensing Data

This notebook demonstrates a temporal perspective on wildfire hazard assessment. It includes data preprocessing, several model evaluation experiments using both temporal (balanced) and random undersampling strategies, and visualisation of the results.

**Requirements:**
- pandas, numpy
- matplotlib, seaborn
- scikit-learn
- imblearn

**Note:** Ensure that your DataFrame (`df`) is initialised and contains the required columns. Some variables (e.g. `burned_area`) must be defined beforehand.

In [ ]:
# 1. Import Libraries
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (roc_auc_score, f1_score, accuracy_score,
                             mean_squared_error, make_scorer)
from imblearn.under_sampling import RandomUnderSampler
from matplotlib.ticker import MultipleLocator


## 2. Data Preprocessing

Before executing this cell, ensure that you have defined the DataFrame `df` and the variable `burned_area` (for example, as a dictionary mapping years to percentage values).

In [ ]:
# 2. Data Preprocessing

# Rename columns for clarity (adjust the list to match your dataframe)
df.columns = [
    'index', 'target_fid', '2001', '2002', '2003', '2004',
    '2005', '2006', '2007', '2008', '2009', '2010',
    '2011', '2012', '2013', '2014', '2015', '2016',
    '2017', '2018', '2019', 'Aspect', 'Elevation', 
    'Land form', 'NDVI', 'Population density', 
    'Rainfall', 'Windspeed', 'Road density', 
    'Slope', 'Stream density', 'TWI',
    '2023', '2022', '2021', '2020', '2019_2'
]

# List of year columns to modify (set any value > 0 to 1)
columns_to_modify = [
    '2001', '2002', '2003', '2004', '2005',
    '2006', '2007', '2008', '2009', '2010', 
    '2011', '2012', '2013', '2014', '2015', 
    '2016', '2017', '2018', '2019', '2020', 
    '2021', '2022', '2023'
]

# Transform values: if cell value > 0, map to 1; otherwise, keep original
df[columns_to_modify] = df[columns_to_modify].applymap(lambda x: 1 if x > 0 else x)

# -----------------------------------------------------------------------------
# Cumulative Burned Area Plot
# -----------------------------------------------------------------------------
# Note: 'burned_area' should be defined (e.g. as a dictionary mapping Year -> Percentage)
cumulative_percentage = burned_area

# Convert the cumulative percentage data to a DataFrame for plotting
cumulative_percentage_df = pd.DataFrame(
    list(cumulative_percentage.items()),
    columns=['Year', 'Cumulative Percentage']
)

# Plot a bar plot of the cumulative percentage of the total burned area
plt.figure(figsize=(10, 4))
sns.barplot(x='Year', y='Cumulative Percentage', data=cumulative_percentage_df, color='grey')
plt.xlabel('Years')
plt.ylabel('Cumulative Percentage of Total Burned Area (%)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 3. Model Evaluation Functions and Experiments

In [ ]:
# Define the years for splitting the data (adjust as needed)
split_years = [2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
               2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
               2020, 2022]

# Cross-validation strategy: 10 folds with 2 repeats
cv = RepeatedKFold(n_splits=10, n_repeats=2, random_state=1)

# Define predictor variables (factors) for the models
factors = [
    'Aspect', 'Elevation', 'Land form', 'NDVI', 'Population density',
    'Rainfall', 'Windspeed', 'Road density', 'Slope', 'Stream density', 'TWI'
]

# -----------------------------------------------------------------------------
# 3.1 Define Model Dictionaries
# -----------------------------------------------------------------------------
# Model set for general evaluation (you can modify parameters as required)
models = {
    'RF': RandomForestClassifier(bootstrap=False, max_depth=6, n_estimators=150, random_state=1),
    'HGB': HistGradientBoostingClassifier(learning_rate=0.2, max_iter=200, min_samples_leaf=40),
    'MLP': Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', MLPClassifier(activation='tanh', hidden_layer_sizes=(12,), max_iter=300, random_state=1))
    ]),
    'KNN': KNeighborsClassifier(n_neighbors=10),
    'LR': LogisticRegression(random_state=1)
}

# Define custom scorers for later use
scoring = {
    'accuracy': 'accuracy',
    'roc_auc': 'roc_auc',
    'f1': 'f1',
    'rmse': make_scorer(mean_squared_error, squared=False)
}

# -----------------------------------------------------------------------------
# Function: create_balanced_sample
# -----------------------------------------------------------------------------
def create_balanced_sample(dataframe, year):
    """
    Create balanced training and testing samples based on a specified year.
    
    Parameters:
      dataframe: Input DataFrame containing a 'Years' column and target 'Wildfire'
      year: Year used to split the data
      
    Returns:
      after_year_balanced: Testing DataFrame (samples with 'Years' > year)
      before_year_balanced: Training DataFrame (samples with 'Years' < year)
    """
    # Positive samples: records where a wildfire event is recorded (Years != 0)
    positive_samples = dataframe[dataframe['Years'] != 0]
    # Negative samples: records with no wildfire event
    negative_samples = dataframe[dataframe['Years'] == 0]
    # Sample negatives to match the number of positives
    negative_samples_sampled = negative_samples.sample(n=positive_samples.shape[0], random_state=1)
    
    # Split positive samples into those before and after the given year
    after_year_samples = positive_samples[positive_samples['Years'] > year]
    before_year_samples = positive_samples[positive_samples['Years'] < year]
    
    # Combine positive samples with a subsample of negatives for balance
    after_year_balanced = pd.concat([
        after_year_samples, 
        negative_samples_sampled.sample(n=after_year_samples.shape[0], random_state=1)
    ])
    before_year_balanced = pd.concat([
        before_year_samples, 
        negative_samples_sampled.sample(n=before_year_samples.shape[0], random_state=1)
    ])
    
    return after_year_balanced, before_year_balanced

# -----------------------------------------------------------------------------
# Function: evaluate_models
# -----------------------------------------------------------------------------
def evaluate_models(model_dict, X_train, y_train, X_test, y_test):
    """
    Evaluate multiple models on the provided training and testing data.
    
    Parameters:
      model_dict: Dictionary of models to evaluate.
      X_train, y_train: Training data.
      X_test, y_test: Testing data.
      
    Returns:
      results: List of dictionaries containing evaluation metrics.
    """
    results = []
    for name, model in model_dict.items():
        # Create a pipeline with scaling and the classifier
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', model)
        ])
        # Train the model
        pipeline.fit(X_train, y_train)
        
        # Predict on test data
        y_pred = pipeline.predict(X_test)
        # If the classifier supports probability estimates, use them
        if hasattr(pipeline.named_steps['classifier'], "predict_proba"):
            y_prob = pipeline.predict_proba(X_test)[:, 1]
        else:
            y_prob = y_pred
        
        # Compute and store metrics
        results.append({
            'model': name,
            'accuracy': accuracy_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_prob),
            'f1': f1_score(y_test, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_test, y_pred))
        })
    return results

# -----------------------------------------------------------------------------
# Function: create_random_undersample
# -----------------------------------------------------------------------------
def create_random_undersample(dataframe, train_size):
    """
    Create balanced training and testing datasets via random undersampling.
    
    Parameters:
      dataframe: Input DataFrame.
      train_size: Proportion of the undersampled data to use for training.
      
    Returns:
      X_train, X_test, y_train, y_test: Split balanced datasets.
    """
    X = dataframe[factors]
    y = dataframe['Wildfire']
    rus = RandomUnderSampler(random_state=1)
    X_res, y_res = rus.fit_resample(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, train_size=train_size, random_state=1)
    return X_train, X_test, y_train, y_test


### Experiment: Temporal (Balanced) Sampling Evaluation

In [ ]:
balanced_samples = {}
evaluation_results_years = []

for year in split_years:
    # Create balanced training (before year) and testing (after year) samples
    train_set, test_set = create_balanced_sample(df, year)
    balanced_samples[year] = {'train': train_set, 'test': test_set}
    
    X_train = train_set[factors]
    y_train = train_set['Wildfire']
    X_test = test_set[factors]
    y_test = test_set['Wildfire']
    
    # Evaluate models on this split
    year_results = evaluate_models(models, X_train, y_train, X_test, y_test)
    # Add additional information to the results
    for result in year_results:
        result['year'] = year
        result['method'] = 'Balanced Sampling'
    evaluation_results_years.extend(year_results)

evaluation_df_years = pd.DataFrame(evaluation_results_years)


### Experiment: Random Undersampling Evaluation

In [ ]:
test_sizes = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.90, 0.99]
evaluation_results_test_sizes = []

for train_size in test_sizes:
    X_train, X_test, y_train, y_test = create_random_undersample(df, train_size)
    test_size_results = evaluate_models(models, X_train, y_train, X_test, y_test)
    for result in test_size_results:
        result['test_size'] = train_size
        result['method'] = 'Random Undersampling'
    evaluation_results_test_sizes.extend(test_size_results)

evaluation_df_test_sizes = pd.DataFrame(evaluation_results_test_sizes)

# Combine both evaluation results for further analysis
evaluation_df = pd.concat([evaluation_df_years, evaluation_df_test_sizes], ignore_index=True)


## 4. Model Evaluation with Timing Metrics

In [ ]:
def evaluate_models_with_timing(model_dict, X_train, y_train, X_test, y_test):
    """
    Evaluate models and record training and prediction times.
    
    Parameters:
      model_dict: Dictionary of models.
      X_train, y_train: Training data.
      X_test, y_test: Testing data.
      
    Returns:
      results: List of dictionaries with metrics and timing information.
    """
    results = []
    for name, model in model_dict.items():
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', model)
        ])
        # Time the model fitting
        start_time = time.time()
        pipeline.fit(X_train, y_train)
        fit_time = time.time() - start_time
        
        # Time prediction on training data
        start_time = time.time()
        y_train_pred = pipeline.predict(X_train)
        if hasattr(pipeline.named_steps['classifier'], "predict_proba"):
            y_train_prob = pipeline.predict_proba(X_train)[:, 1]
        else:
            y_train_prob = y_train_pred
        train_pred_time = time.time() - start_time
        
        # Time prediction on testing data
        start_time = time.time()
        y_test_pred = pipeline.predict(X_test)
        if hasattr(pipeline.named_steps['classifier'], "predict_proba"):
            y_test_prob = pipeline.predict_proba(X_test)[:, 1]
        else:
            y_test_prob = y_test_pred
        test_pred_time = time.time() - start_time
        
        # Record training set metrics
        results.append({
            'model': name,
            'set': 'train',
            'accuracy': accuracy_score(y_train, y_train_pred),
            'roc_auc': roc_auc_score(y_train, y_train_prob),
            'f1': f1_score(y_train, y_train_pred),
            'rmse': np.sqrt(mean_squared_error(y_train, y_train_pred)),
            'fit_time': fit_time,
            'pred_time': train_pred_time
        })
        # Record testing set metrics
        results.append({
            'model': name,
            'set': 'test',
            'accuracy': accuracy_score(y_test, y_test_pred),
            'roc_auc': roc_auc_score(y_test, y_test_prob),
            'f1': f1_score(y_test, y_test_pred),
            'rmse': np.sqrt(mean_squared_error(y_test, y_test_pred)),
            'fit_time': fit_time,
            'pred_time': test_pred_time
        })
    return results

# Evaluate with timing using temporal sampling
evaluation_results_years_timing = []
for year in split_years:
    train_set, test_set = create_balanced_sample(df, year)
    X_train = train_set[factors]
    y_train = train_set['Wildfire']
    X_test = test_set[factors]
    y_test = test_set['Wildfire']
    
    year_results = evaluate_models_with_timing(models, X_train, y_train, X_test, y_test)
    for result in year_results:
        result['year'] = year
        result['method'] = 'Balanced Sampling'
    evaluation_results_years_timing.extend(year_results)

evaluation_df_years_timing = pd.DataFrame(evaluation_results_years_timing)

# Evaluate with timing using random undersampling
evaluation_results_test_sizes_timing = []
for train_size in test_sizes:
    X_train, X_test, y_train, y_test = create_random_undersample(df, train_size)
    test_size_results = evaluate_models_with_timing(models, X_train, y_train, X_test, y_test)
    for result in test_size_results:
        result['test_size'] = train_size
        result['method'] = 'Random Undersampling'
    evaluation_results_test_sizes_timing.extend(test_size_results)

evaluation_df_test_sizes_timing = pd.DataFrame(evaluation_results_test_sizes_timing)
evaluation_df_timing = pd.concat([evaluation_df_years_timing, evaluation_df_test_sizes_timing], ignore_index=True)


## 5. Visualisation of Evaluation Metrics

In [ ]:
# Define the metrics to visualise
metrics = ['accuracy', 'roc_auc', 'f1', 'rmse', 'fit_time', 'pred_time']
# Define method labels for plotting
methods = ['Temporal sampling', 'Random sampling']

for metric in metrics:
    # -------------------------------
    # Plot for the training set
    # -------------------------------
    fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)
    fig.suptitle(f'{metric.capitalize()} for Train Set')
    
    for ax, method in zip(axes, methods):
        # Select the subset based on the method and training set
        subset = evaluation_df_timing[(evaluation_df_timing['method'] == method) & (evaluation_df_timing['set'] == 'train')]
        if method == 'Temporal sampling':
            x_var = 'year'
            xlabel = 'Year'
            ax.set_xlim(2002, 2022)
            ax.xaxis.set_major_locator(MultipleLocator(2))
        else:
            x_var = 'test_size'
            xlabel = 'Train Size (%)'
        
        for model_name in subset['model'].unique():
            model_subset = subset[subset['model'] == model_name]
            ax.plot(model_subset[x_var], model_subset[metric],
                    marker='o', label=model_name, linewidth=1, markersize=3)
        
        ax.set_title(method)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(metric.capitalize())
        ax.legend()
        ax.grid(True)
    
    plt.savefig(f'evaluation_{metric}_train.png')
    plt.show()
    
    # -------------------------------
    # Plot for the testing set
    # -------------------------------
    fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)
    fig.suptitle(f'{metric.capitalize()} for Test Set')
    
    for ax, method in zip(axes, methods):
        subset = evaluation_df_timing[(evaluation_df_timing['method'] == method) & (evaluation_df_timing['set'] == 'test')]
        if method == 'Temporal sampling':
            x_var = 'year'
            xlabel = 'Year'
            ax.set_xlim(2002, 2022)
            ax.xaxis.set_major_locator(MultipleLocator(2))
        else:
            x_var = 'test_size'
            xlabel = 'Train Size (%)'
        
        for model_name in subset['model'].unique():
            model_subset = subset[subset['model'] == model_name]
            ax.plot(model_subset[x_var], model_subset[metric],
                    marker='o', label=model_name, linewidth=1, markersize=3)
        
        ax.set_title(method)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(metric.capitalize())
        ax.legend()
        ax.grid(True)
    
    plt.savefig(f'evaluation_{metric}_test.png')
    plt.show()


## 6. Additional Analysis: Histogram Visualisation of Wildfire Susceptibility

This section creates balanced samples for a specific year and plots histograms for selected models.

In [ ]:
# Create balanced samples for a specific analysis year (e.g. 2012)
year_for_analysis = 2012
after_year_balanced, before_year_balanced = create_balanced_sample(df, year_for_analysis)

# Prepare training and testing data using the balanced sample
X_train = before_year_balanced[factors]
y_train = before_year_balanced['Wildfire']
X_test = after_year_balanced[factors]
y_test = after_year_balanced['Wildfire']

# Define a dictionary of models for further analysis (parameters can be adjusted)
models_analysis = {
    'RF': RandomForestClassifier(bootstrap=False, max_depth=9, n_estimators=25, random_state=1),
    'HGB': HistGradientBoostingClassifier(min_samples_leaf=190, max_iter=90, max_depth=7, max_leaf_nodes=33, learning_rate=0.15),
    'MLP': Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', MLPClassifier(activation='tanh', hidden_layer_sizes=(12,), max_iter=300, random_state=1))
    ]),
    'KNN': KNeighborsClassifier(n_neighbors=34, p=2),
    'LR': LogisticRegression(random_state=1),
    'SVM': SVC(C=100, random_state=1, probability=True)
}

# Train each model and add the predicted probabilities to the dataframe
for name, model in models_analysis.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', model)
    ])
    pipeline.fit(X_train, y_train)
    # Predict on the entire dataframe using the selected factors
    prob = pipeline.predict_proba(df[factors])[:, 1]
    df[name] = prob

# Plot histograms of the wildfire susceptibility index for selected models.
# (Note: Replace 'tomporal' with your actual dataframe name if necessary.)
selected_models = ['RF', 'HGB', 'MLP', 'KNN', 'LR']
for model_name in selected_models:
    plt.figure(figsize=(8, 4))
    sns.histplot(
        data=df.query('Years > 2012 and Years != 2018'),
        x=model_name,
        hue='Years',
        common_norm=True,
        palette='jet_r',
        stat='density',
        multiple='stack',
        bins=34
    )
    plt.xlabel('Wildfire Susceptibility Index', size=12)
    plt.ylabel('Wildfire Density', size=12)
    plt.xlim(0, 1)
    plt.ylim(0, 5)
    plt.tight_layout()
    plt.savefig(f'{model_name}_histo_temporal_scaled_common.png', dpi=500)
    plt.show()


## 7. Final Visualisation: Bar Plots of Evaluation Metrics

This final section replaces some method names for clarity, calculates processing time, and creates bar plots for selected evaluation metrics.

In [ ]:
# Replace method names for clarity
evaluation_df_timing['method'] = evaluation_df_timing['method'].str.replace('Random Undersampling', 'Random sampling')
evaluation_df_timing['method'] = evaluation_df_timing['method'].str.replace('Balanced Sampling', 'Temporal sampling')

# Calculate processing time as the sum of fit and prediction times
evaluation_df_timing['Processing time (s)'] = evaluation_df_timing['fit_time'] + evaluation_df_timing['pred_time']

# Filter results for the test set only
eval_df_test = evaluation_df_timing[evaluation_df_timing['set'] == 'test']

# Melt the DataFrame for easier plotting of metrics
df_melted = eval_df_test.melt(
    id_vars=['model', 'method'],
    value_vars=['accuracy', 'roc_auc', 'f1', 'rmse', 'Processing time (s)'],
    var_name='metric',
    value_name='value'
)

# Filter for selected models
df_melted = df_melted.query('model in ["RF", "HGB", "MLP", "KNN", "LR"]')

# Create subplots for each metric
metrics_to_plot = ['accuracy', 'roc_auc', 'f1', 'rmse', 'Processing time (s)']
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 8), constrained_layout=True)
axes = axes.flatten()

# Define pastel colours for the bars
colors = ["#f6bc66", "#f49097"]

for i, metric in enumerate(metrics_to_plot):
    sns.barplot(
        data=df_melted[df_melted['metric'] == metric],
        x='model', y='value', hue='method', ax=axes[i],
        palette=colors, linewidth=1, edgecolor=".4"
    )
    axes[i].set_xlabel('Model')
    axes[i].set_ylabel(metric.capitalize())
    axes[i].legend(title='Sampling', loc='lower left')
    # For some metrics, adjust the y-axis limits if required (example shown for the first row)
    if i < 3:
        axes[i].set_ylim(0.5, 1)

# Adjust one of the legends if necessary
axes[4].legend(title='Sampling', loc='upper left')

# Remove any unused subplot
if len(metrics_to_plot) < len(axes):
    for j in range(len(metrics_to_plot), len(axes)):
        fig.delaxes(axes[j])

plt.savefig('CV_Metrics.png', dpi=500)
plt.show()
